In [2]:
import pandas as pd
import sqlite3
import data_files_manipulation as dfm

# NHL Game Data
---------
## This collection of hockey data files was originally found on Kaggle:
### https://www.kaggle.com/datasets/martinellis/nhl-game-data

Some unwanted files were dropped, though most remain. [See the process for trimming the game_plays file](trim_game_plays.ipynb)
 
[Here is a diagram of the table relationships](table_relationships.JPG)

### Data Preparation
---
The files have been added to the repo in a compressed file. Run the cells below to unzip folder and move them to the project directory.

In [9]:
dfm.unzip_all_files()

In [6]:
dfm.move_all_files('nhl_data_files_1','..')
dfm.move_all_files('nhl_data_files_2','..')
dfm.move_all_files('player_stats_data_files','..')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'nhl_data_files_1'

### Create the SQL database
-----
This function will take a list of csv files, and compile a single SQL database, each csv file as a table. 

This is particularly useful when there are many different files that need to be connected in multiple ways before analysis can happen. Using a SQL query, a DataFrame with only the wanted information can be found and then manipulated using pandas.

In [10]:
def create_sql_database(list_of_files, db_name):
    conn = sqlite3.connect(db_name)
    for file in list_of_files:
        df = pd.read_csv(file, low_memory=False)
        df.to_sql(name = file.replace('.csv',''), con = conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

In [11]:
nhl_data_files = ['game.csv','game_goals.csv', 'game_penalties.csv','game_plays_trimmed.csv','game_plays_players.csv', 'game_shifts.csv','game_skater_stats.csv','game_teams_stats.csv','player_info.csv','team_info.csv']

### Construct the initial SQL database for the project
---
With just one function below, a new NHL_data database is created to use for this project's analysis.

In [12]:
create_sql_database(nhl_data_files, 'NHL_data.db')

Verify that everything is successful by printing the table names below:

In [13]:
table_test_query = """
    SELECT name FROM sqlite_master 
    WHERE type='table';
    """
conn = sqlite3.connect('NHL_data.db')
cur = conn.cursor()
cur.execute(table_test_query)
print("List of tables\n")
# printing all tables list
print(cur.fetchall())

List of tables

[('game',), ('game_goals',), ('game_penalties',), ('game_plays_trimmed',), ('game_plays_players',), ('game_shifts',), ('game_skater_stats',), ('game_teams_stats',), ('player_info',), ('team_info',)]
